# Bostrom supply simulation
---

## A/B simulations params (depricated)


| Param name | Simulation A | Simulation B|
|:---|---:|---:|
|Liquid Tokens (T)|200,000,000,000,000|200,000,000,000,000|
|Frozen Tokens (F)|700,000,000,000,000|700,000,000,000,000|
|Vested Tokens (L)|100,000,000,000,000|100,000,000,000,000|
|Initial inflation |5.00|5.00|
|Amper Mint Rate init|50.00|50.00|
|Volt Mint Rate init|100.00|100.00|
|Inflation rate change|13.00|13.00|
|Inflation min|1.00|1.00|
|Inflation max|15.00|15.00|
|Goal vested|80.00|80.00|
|Volt halving|1,000,000.00|1,000,000.00|
|Resource token init price|100,000,000.00|100,000,000.00|

In [ ]:
import time
import itertools

# Standard libraries: https://docs.python.org/3/library/
import math

# Analysis and plotting modules
import pandas as pd

# cadCAD configuration modules
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment

# cadCAD simulation engine modules
from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor

from collections import Counter
from cadCAD import configs

# custom functions modules

from utils.psub import partial_state_update_blocks

from utils.plots import linear_plot, scatter_plot, df_preparator

## Initial state

In [ ]:
sim_period = 10          # amount of years for simulating
timesteps_per_year = 365        # units per year(days`

In [ ]:
initial_state = {
    'liquid_boot': 200_000_000_000_000,
    'frozen_boot': 700_000_000_000_000,
    'vested_boot': 100_000_000_000_000,
    'amper': 0,
    'volt': 0,
    'investmint_max_period': math.floor(timesteps_per_year / 2),
    'boot_inflation_rate': 0.10,
    'mint_rate_amper': 100,
    'mint_rate_volt': 100,
    'cyberlinks': 0,
    'agents_count': 250,
    'capitalization_per_agent': 1
}

## Params for simulating

In [ ]:
vesting_speed = [3, 6, 12, 18, 24, 30, 36]
unvesting_speed = [72, 60, 48, 36, 30, 24]

parameter_sweep = list(itertools.product(vesting_speed, unvesting_speed))

vesting_speed = [x[0] for x in parameter_sweep]
unvesting_speed = [x[1] for x in parameter_sweep]

In [ ]:
system_params = {
    'boot_inflation_rate_change_annual': [0.07], # maximum inflation rate change per year
    'boot_inflation_min': [0.05], # minimum percent of the inflation
    'boot_inflation_max': [0.20], # maximum percent of the inflation
    'boot_bonded_share_target': [0.70], # desireable ratio between vested tokens and tokens supply
    'timesteps_per_year': [timesteps_per_year], # units per year(minutes)
    'vesting_speed': vesting_speed, # amount of months to vest all liquid tokens
    'unvesting_speed': unvesting_speed, # amount of months to uninvest all vested tokens
    'base_halving_period_amper': [12_000_000 * 6],
    'base_halving_period_volt': [12_000_000 * 6],
    'mint_rate_amper_init': [100], # initial mint rate for Amperes
    'mint_rate_volt_init': [100], # initial mint rate for Voltes
    'base_investmint_amount_amper': [100_000_000], # initial cost for resource token in 1 cycle
    'base_investmint_amount_volt': [100_000_000], # initial cost for resource token in 1 cycle
    'base_investmint_preiod_amper': [math.floor(timesteps_per_year / 12)],
    'base_investmint_preiod_volt': [math.floor(timesteps_per_year / 12)],
    'investmint_max_period_init': [math.floor(timesteps_per_year / 2)],
    'boot_bonded_share_target': [0.7],
    'extra_links': [100],
    'guaranted_links': [100],
    'agents_count_at_activation': [1000],
    'start_capitalization_per_agent': [1]
}

In [ ]:
del configs[:]
experiment = Experiment()

sim_config = config_sim({
    'N': 1,
    'T': range(int(math.ceil(sim_period * timesteps_per_year))),
    'M': system_params
})

experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)

In [ ]:
exec_context = ExecutionContext()

simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute()

In [ ]:
start_time = time.time()
simulation_result = pd.DataFrame(raw_result)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()

In [ ]:
df = simulation_result.copy()
df['boot_supply'] = df['liquid_boot'] + df['frozen_boot'] + df['vested_boot']
df['cap_in_eth'] = df['capitalization_per_agent'] * df['agents_count']
df

In [ ]:
# save local model file for power bi
# import pyarrow as pa
# import pyarrow.parquet as pq
# table = pa.Table.from_pandas(df)
# pq.write_table(table, 'simulation.parquet')

In [ ]:
plot_df = df_preparator(df)

## Inflation

In [ ]:
scatter_plot(plot_df, "boot_inflation_rate", render='sns')

## Supply

In [ ]:
scatter_plot(plot_df, 'boot_supply', render='sns')

## Vested

In [ ]:
scatter_plot(plot_df, 'capitalization_per_agent', render='sns')

## Ampler tokens supply

In [ ]:
scatter_plot(plot_df, 'amper', render='sns')

## Volt tokens supply

In [ ]:
scatter_plot(plot_df, 'volt', render='sns')

## Mint Rate Volter-Amper

In [ ]:
linear_plot(plot_df, 'mint_rate_volt', render='sns')

## Max vesting time

In [ ]:
linear_plot(plot_df, 'maxVestingTime', render='sns')

## Ratio between amper supply and volt supply

In [ ]:
scatter_plot(plot_df, 'AmPrice', render='sns')

## Cyberlinks

In [ ]:
scatter_plot(plot_df, 'CL', render='sns')

## Agents amount

In [ ]:
scatter_plot(plot_df, 'agents_count', render='sns')

## Cap in ETH

In [ ]:
scatter_plot(plot_df, 'cap_in_eth', render='sns')

In [ ]:
print("--- %s seconds ---" % (time.time() - start_time))